In [5]:
# import streamlit as st
# import pandas as pd
# import plotly.express as px

# df = pd.read_csv('911.csv')
# df.drop_duplicates(inplace = True , ignore_index=True)
# df.reset_index(drop = True, inplace = True)
# df = df.dropna(subset=['twp'])

# # Get unique township values from the 'twp' column
# unique_townships = df['twp'].unique()
# # Create a dictionary mapping township to zip code
# township_to_zip_mapping = {twp: None for twp in unique_townships}
# # Fill the dictionary with zip codes
# for twp in unique_townships:
#     zip_code = None  # Initialize zip_code as None
#     rows_with_twp = df[df['twp'] == twp]  # Filter rows with the same twp
#     for index, row in rows_with_twp.iterrows():
#         if pd.notna(row['zip']):  # Check if zip is not NaN
#             zip_code = row['zip']
#             break  # Stop searching once a non-null zip is found
#     township_to_zip_mapping[twp] = zip_code
# # Fill missing zip codes based on township
# df['zip'] = df.apply(lambda row: township_to_zip_mapping.get(row['twp'], row['zip']), axis=1)

# df = df.drop(columns=['e'])
# df['zip'] = df['zip'].astype(int).astype(str)

# df['Department'] = df['title'].apply(lambda title: title.split(':')[0])
# df['Reason'] = df['title'].apply(lambda title: title.split(':')[1])
# df['timeStamp'] = pd.to_datetime(df['timeStamp'])
# df['Date'] = df['timeStamp'].dt.date
# df['Hour'] = df['timeStamp'].apply(lambda time: time.hour)
# df['Day'] = df['timeStamp'].dt.day
# df['nDay_of_Week'] = df['timeStamp'].apply(lambda time: time.dayofweek)
# dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
# df['Day_of_Week'] = df['nDay_of_Week'].map(dmap)
# df['Month'] = df['timeStamp'].apply(lambda time: time.month)
# df['Year'] = df['timeStamp'].apply(lambda time: time.year)

# df.to_csv('M911.csv')

In [9]:
import streamlit as st
import pandas as pd
import plotly.express as px

In [11]:
df = pd.read_csv('M911.csv')

df[['lat', 'lng']]

,lat,lng
0,40.297876,-75.581294
1,40.258061,-75.264680
2,40.121182,-75.351975
3,40.116153,-75.343513
4,40.251492,-75.603350
...,...,...
662984,40.157956,-75.348060
662985,40.136306,-75.428697
662986,40.013779,-75.300835
662987,40.121603,-75.351437


In [30]:
%%writefile 911_dashboard.py

import streamlit as st
import pandas as pd
import plotly.express as px

st.set_page_config(layout='wide',
                  page_title = '911 Call Data Analysis Dashboard')

df = pd.read_csv('M911.csv')

def main_page():
    # Set page title
    st.title("911 Call Data Analysis Dashboard")

    st.image('Police.png')

    # Project Overview
    st.markdown("""
                
    ## Overview

    ### Understanding the Background

    * Montgomery County

    Montgomery County, locally also referred to as Montco, is a county located in the Commonwealth of Pennsylvania. As of the 2010 census, the population was 799,874, making it the third-most populous county in Pennsylvania, after Philadelphia and Allegheny Counties. The county seat is Norristown. Montgomery County is very diverse, ranging from farms and open land in Upper Hanover to densely populated rowhouse streets in Cheltenham.

    * 911 Calls

    Created by Congress in 2004 as the 911 Implementation and Coordination Office (ICO), the National 911 Program is housed within the National Highway Traffic Safety Administration at the U.S. Department of Transportation and is a joint program with the National Telecommunication and Information Administration in the Department of Commerce.

    ### Goal:

    * Locations from which 911 calls are most frequent
    * Time daily, month, weekly patterns of 911 calls
    * Major Causes of 911 calls


    **This analysis will help to deploy more agents in specific location and save/help people at right time**

    ---
    ----

    ### The Data

    `Acknowledgements`: Data provided by  <a href='http://www.pieriandata.com'>montcoalert.org</a>

    we will be analyzing some 911 call data from [Kaggle](https://www.kaggle.com/mchirico/montcoalert). The data contains the following fields:

    Column | Definition
    --- | -----------
    lat | Latitude
    lng | Longitude
    desc | Description of the Emergency Call
    zip | Zipcode
    title | Title of Emergency
    timeStamp | YYYY-MM-DD HH:MM:SS
    twp | Township
    addr | Address
    e | Dummy variable (always 1)


                """)

    st.markdown("""
    ---
    ----
    """)

    datasample = st.checkbox('Show Data Sample', False, key =1)
    if datasample:
        st.dataframe(df.head())

def page2():
    df = pd.read_csv('M911.csv')
    # Sidebar options
    st.sidebar.header("Dashboard Options")

    # Filter by Department
    selected_department = st.sidebar.selectbox("Select Department", ['All'] + df['Department'].unique().tolist())
    if selected_department != 'All':
        df = df[df['Department'] == selected_department]

    # Filter by Reason
    selected_reason = st.sidebar.selectbox("Select Reason", ['All'] + df['Reason'].unique().tolist())
    if selected_reason != 'All':
        df = df[df['Reason'] == selected_reason]

    # Visualizations
    st.header("Incident Distribution by Department and Reason")
    fig_reasons = px.histogram(df, x='Department', color='Reason', labels={'Department': 'Department', 'Reason': 'Reason'},
                            title='Distribution of Incidents by Department and Reason')
    st.plotly_chart(fig_reasons)

    st.header("Incident Counts by Date")
    # Group data by date and calculate incident counts
    byDate = df.groupby('Date').count()['twp'].reset_index()
    # Create an interactive line plot using Plotly Express
    fig_date = px.line(byDate, x='Date', y='twp', labels={'Date': 'Date', 'twp': 'Incident Counts'},
                    title='Incident Counts by Date')
    st.plotly_chart(fig_date)
    
def page3():
    df = pd.read_csv('M911.csv')
    # Additional visualizations can be added here
    df.rename(columns={'lng': 'lon'}, inplace=True)
    map_data = df[['lat', 'lon']]

    st.map(map_data)

pages = { '1':main_page,
          '2':page2,
          '3': page3
}    

select_page = st.sidebar.selectbox('select ur page',pages.keys())
pages[select_page]()


# Show footer
st.sidebar.text("Created with Streamlit by Ahmed NasrElDin & Gehad Samir")

Overwriting 911_dashboard.py


In [31]:
! streamlit run 911_dashboard.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.1.3:8501

^C
  Stopping...
